# Downloading Exoplanet Data Solutions

***

This notebook contains solutions to the exercises described in the Exercises section of `downloading_exoplanet_data.ipynb`.

We begin with the same set of imports as the tutorial notebook.

In [ ]:
import ast
import sys
import os
import json
import pprint
import urllib.request
from urllib.parse import quote as urlencode

import numpy as np
import requests
from astropy.table import Table
from astroquery.mast import Observations

# Exercise 1
*Use the MAST API to sort Swift data on WASP-12 b by PI and download it.*

We begin with much of the same procedure as the tutorial notebook.

In [ ]:
object_of_interest = 'WASP-12 b'

resolver_request = {'service':'Mast.Name.Lookup',
                     'params':{'input':object_of_interest,
                               'format':'json'},
                     }

# encoding the request as a json string
req_string = json.dumps(resolver_request)
req_string = urlencode(req_string)

In [ ]:
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
object_of_interest = 'WASP-12 b'

resolver_request = {'service':'Mast.Name.Lookup',
                     'params':{'input':object_of_interest,
                               'format':'json'},
                     }

# encoding the request as a json string
req_string = json.dumps(resolver_request)
req_string = urlencode(req_string)

In [ ]:
# retrieve our system's Python version for the request. 
version = ".".join(map(str, sys.version_info[:3]))

# create HTTP Header Variables
headers = {"Content-type": "application/x-www-form-urlencoded",
           "Accept": "text/plain",
           "User-agent":"python-requests/"+version}


In [ ]:
# base API url
request_url='https://mast.stsci.edu/api/v0/invoke'    

# perform the HTTP request
resp = requests.post(request_url, data="request="+req_string, headers=headers)

In [ ]:
resolved_object = resp.json()

pp.pprint(resolved_object)

In [ ]:
obj_ra = resolved_object['resolvedCoordinate'][0]['ra']
obj_dec = resolved_object['resolvedCoordinate'][0]['decl']

obj_ra, obj_dec

In [ ]:
mast_request = {'service':'Mast.Caom.Cone',
                'params':{'ra':obj_ra,
                          'dec':obj_dec,
                          'radius':0.2},
                'format':'json',
                'pagesize':2000,
                'page':1,
                'removenullcolumns':True,
                'removecache':True}


# encoding the request as a json string
req_string = json.dumps(mast_request)
req_string = urlencode(req_string)

# perform the HTTP request
resp = requests.post(request_url, data="request="+req_string, headers=headers)

# decode the HTTP result
mast_data = resp.json()


print(mast_data.keys())
print("Query status:",mast_data['status'])

In [ ]:
mast_data_table = Table()

col_type_list = [(field['name'], field['type']) for field in mast_data['fields']]

for col, atype in col_type_list:
    
    # recast data types
    if atype=="string":
        atype="str"
    if atype=="boolean":
        atype="bool"
    
    # create column in Table corresponding to the mast_data field
    mast_data_table[col] = np.array([x.get(col, None) for x in mast_data['data']],dtype=atype)
    
print(mast_data_table)

In [ ]:
mast_data_table.columns

Instead of next sorting by `t_min`, though, we sort by PI.

In [ ]:
mast_data_table.sort('proposal_pi')

In [ ]:
print(mast_data_table)

In [ ]:
recent_index = -1
interesting_observation = mast_data_table[mast_data_table["obs_collection"] == "SWIFT"][recent_index]
print("Observation:",
      [interesting_observation[x] for x in ['dataproduct_type', 'obs_collection', 'instrument_name']])

In [ ]:
obsid = interesting_observation['obsid']

product_request = {'service':'Mast.Caom.Products',
                  'params':{'obsid':obsid},
                  'format':'json',
                  'pagesize':100,
                  'page':1}   

# encoding the request as a json string
req_string = json.dumps(product_request)
req_string = urlencode(req_string)

# perform the HTTP request
resp = requests.post(request_url, data="request="+req_string, headers=headers)

# dfrecode the HTTP result
obs_products = resp.json()

print("Number of data products:", len(obs_products["data"]))
print("Product information column names:")
pp.pprint(obs_products['fields'])

In [ ]:
pp.pprint([x.get('productType',"") for x in obs_products["data"]])

In [ ]:
sci_prod_arr = [x for x in obs_products['data'] if x.get("productType", None) == 'SCIENCE']
science_products = Table()

col_type_list = [(field['name'], field['type']) for field in obs_products['fields']]

for col, atype in col_type_list:
    
    # recast data types
    if atype=="string":
        atype="str"
    if atype=="boolean":
        atype="bool"
    if atype == "int":
        atype = "float" # array may contain nan values, and they do not exist in numpy integer arrays
        
    # Create column in Table corresponding to the mast_data field
    science_products[col] = np.array([x.get(col,None) for x in sci_prod_arr],dtype=atype)

print("Number of science products:",len(science_products))
print(science_products)

In [ ]:
download_url = 'https://mast.stsci.edu/api/v0.1/Download/file?'

for row in science_products:     

    # Make file path
    out_path = os.path.join("mastFiles", row['obs_collection'], row['obs_id'])
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    out_path = os.path.join(out_path, os.path.basename(row['productFilename']))
        
    # Download the data
    payload = {"uri":row['dataURI']}
    resp = requests.get(download_url, params=payload)
    
    # Save to file
    with open(out_path,'wb') as f:
        f.write(resp.content)
        
    # Check for file 
    if not os.path.isfile(out_path):
        print("ERROR: " + out_path + " failed to download.")
    else:
        print("COMPLETE: ", out_path)

In [ ]:
ls mastFiles/SWIFT/00032242001/

# Exercise 2

*Query the latest light curve of WASP-12 b.*
*Question: Is this too close to the actual tutorial?*

We can fetch light curves from the TESS mission.

In [ ]:
search_radius = ".02 deg"

planet_name = 'WASP-12 b'
obs_table = Observations.query_object(planet_name,radius=search_radius)

# only print the first 10 results for clarity
print(obs_table[:10])  

In [ ]:
obs_table

To get the latest results, we sort by t_min.

In [ ]:
obs_table.sort('t_min')

However, we want to make sure that we aren't selecting an observation with a `NaN` value for `t_min`.

In [ ]:
not_nan_mask = ~np.isnan(obs_table['t_min'])

obs_table = obs_table[not_nan_mask]

In [ ]:
obs_table

In [ ]:
obs_table_qlp = obs_table[obs_table['project']=='TESS']

In [ ]:
recent_index = -1
data_products_by_obs = Observations.get_product_list(obs_table_qlp[recent_index])
print(data_products_by_obs) 

In [ ]:
data_products_by_obs.columns

In [ ]:
# select first observation collection
obs_collection = data_products_by_obs['obs_collection'][0]
obs_id = data_products_by_obs['obs_id'][0]

# query and get product list for this collection
single_obs = Observations.query_criteria(obs_collection=obs_collection, obs_id=obs_id)
data_products = Observations.get_product_list(single_obs)

# download data products
manifest = Observations.download_products(data_products, productType="SCIENCE")

# Exercise 3

In [ ]:
planet_name = 'BD+47 2936 b'

In [ ]:
planet_name_formatted = planet_name.replace(' ', '%20')

request_name_string = f'exoplanets/identifiers/?name={planet_name_formatted}'

In [ ]:
request_name_url = "https://exo.mast.stsci.edu/api/v0.1/" + request_name_string
print(request_name_url)

In [ ]:
names = urllib.request.urlopen(request_name_url).read()
names

Now, we note that the canonical name for this planet is HAT-P-11 b. We use this name for our data request.

In [ ]:
planet_name = 'HAT-P-11 b'
planet_name_formatted = planet_name.replace(' ', '%20')

In [ ]:
dict_str = names.decode("UTF-8")

In [ ]:
dict_str = dict_str.replace('null', 'None')
dict_str

In [ ]:
name_matches = ast.literal_eval(dict_str)

In [ ]:
name_matches

In [ ]:
# construct request string
request_spectra_string = f'spectra/{planet_name_formatted}/filelist/'
request_spectra_url = "https://exo.mast.stsci.edu/api/v0.1/" + request_spectra_string

# send the request
spectra_result = urllib.request.urlopen(request_spectra_url).read()

# decode the result and make it Pythonic
dict_str = spectra_result.decode("UTF-8")
dict_str = dict_str.replace('null', 'None')

# evaluate the result
spectra = ast.literal_eval(dict_str)
spectra

In [ ]:
# construct request string
filename = spectra['filenames'][0]
request_file_string = f'spectra/{planet_name_formatted}/file/{filename}'
request_file_url = "https://exo.mast.stsci.edu/api/v0.1/" + request_file_string

# send the request
spectra_result = urllib.request.urlopen(request_file_url).read()

# decode the result and make it Pythonic
downloaded_file = spectra_result.decode("UTF-8")
downloaded_file

In [ ]:
with open(filename, 'w') as f:
    f.write(downloaded_file)